# Tokenize and Train

### Author 
Stephen Lee

### Goal
Classify news source based on the article text. Training data: 
- Fox News
- Vox News
- PBS News

### Date 
4.8.19

## Read Data

In [1]:
from keras.preprocessing.text import Tokenizer 
from keras.preprocessing.sequence import pad_sequences 
from keras.models import Sequential
from keras.layers import GRU, Dense, Bidirectional, LSTM, Activation
from keras.utils import to_categorical

import os 
import math 
import pandas as pd 

from sklearn.model_selection import train_test_split
from sklearn import metrics

Using TensorFlow backend.
/home/smlee_981/anaconda3/lib/python3.5/site-packages/sklearn/utils/fixes.py:313: FutureWarning: numpy not_equal will not check object identity in the future. The comparison did not return the same result as suggested by the identity (`is`)) and will change.
  _nan_object_mask = _nan_object_array != _nan_object_array


In [2]:
FOLDER_READ = '/home/smlee_981/data'
FILE = 'clean_article_df.csv'
ARTICLE_LENGTH = 500    # max length for an article

In [3]:
os.getcwd()

'/home/smlee_981'

In [4]:
os.chdir(FOLDER_READ)

In [5]:
os.listdir()

['clean_article_df.csv', 'glove.840B.300d.txt']

In [7]:
df_all = pd.read_csv(FILE, sep='|').drop('Unnamed: 0', axis=1).drop('article', axis=1)
df_all.head()

article id source                                     clean_articles  \
0  fox_politics_166    Fox  <br>\nFormer New Jersey Gov. Chris Christie sa...   
1  fox_politics_390    Fox  FILE--In this July 28, 2016 file photo, Sen. B...   
2  fox_politics_423    Fox  Howard Kurtz: How Michael Cohen, Democrats sto...   
3  fox_politics_102    Fox  Student Union: Make UC Berkeley a sanctuary ca...   
4  fox_politics_492    Fox  President Trump’s health care executive order:...   

   targets  
0        2  
1        2  
2        2  
3        2  
4        2

## Relabel targets from 0 - 2 for one-hot vectors

In [8]:
df_all['targets'] = df_all['targets'].replace(3,0)
df_all.groupby('source').mean()

targets
source         
Fox           2
PBS           0
Vox           1

# First dataset
#### Bootstrap data for balance

In [9]:
df_all.groupby('source').count()

article id  clean_articles  targets
source                                     
Fox            661             661      661
PBS           1739            1739     1739
Vox           1027            1027     1027

## Add more Fox and Vox

In [10]:
fox = df_all[df_all['source'] == 'Fox']
vox = df_all[df_all['source'] == 'Vox']
pbs = df_all[df_all['source'] == 'PBS']

In [13]:
# start with the full dataset and append with 
# shorter targets
pbs_fox_diff = len(pbs) - len(fox)
pbs_vox_diff = len(pbs) - len(vox)

fox_append = fox.sample(pbs_fox_diff, replace=True)
vox_append = vox.sample(pbs_vox_diff)

df_first = df_all.copy()
df_first = df_first.append(fox_append, ignore_index=True)
df_first = df_first.append(vox_append, ignore_index=True)
df_first.groupby('source').count()

article id  clean_articles  targets
source                                     
Fox           1739            1739     1739
PBS           1739            1739     1739
Vox           1739            1739     1739

## Split into test and training

In [14]:
train_df, test_df = train_test_split(df_first, test_size=0.1)
train_df.head()

article id source  \
4571  vox_politics_1803    Vox   
4889   vox_politics_246    Vox   
4160   fox_politics_890    Fox   
2916   pbs_politics_777    PBS   
4747  vox_politics_1931    Vox   

                                         clean_articles  targets  
4571  On January 31, Reuters released a survey that ...        1  
4889  Sen. Elizabeth Warren (D-MA) isn’t ready to ce...        1  
4160  Michael Cohen testifies behind closed doors on...        2  
2916    Democratic senators plan to press Attorney G...        0  
4747  Buried in Republicans’ tax bill, in a provisio...        1

In [15]:
test_df.head()

article id source  \
2990  pbs_politics_1697    PBS   
411   fox_politics_1003    Fox   
1209    vox_politics_92    Vox   
1925  pbs_politics_1041    PBS   
3166  pbs_politics_1344    PBS   

                                         clean_articles  targets  
2990  The 24-hour news cycle is filled with politics...        0  
411   Pelosi works on Democratic leadership term-lim...        2  
1209  In an interview that aired Sunday on 60 Minute...        1  
1925    White House press secretary Sarah Huckabee S...        0  
3166  President Donald Trump says he has a “great lo...        0

### Check for similarity between test and training

In [16]:
test_df.groupby('source').count()

article id  clean_articles  targets
source                                     
Fox            173             173      173
PBS            156             156      156
Vox            193             193      193

In [17]:
train_df.groupby('source').count()

article id  clean_articles  targets
source                                     
Fox           1566            1566     1566
PBS           1583            1583     1583
Vox           1546            1546     1546

## Get Embeddings, Build Functions

In [18]:
from tqdm import tqdm
import numpy as np

In [19]:
# glove embeddings and data are in same folder

EMBEDS = 'glove.840B.300d.txt'

embeddings_index = {}
 
with open(EMBEDS, encoding='utf8') as embed:
    for line in tqdm(embed):
        values = line.split(' ')
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs
        
print("Found {n} word vectors".format(n=len(embeddings_index)))

2196017it [03:08, 11651.71it/s]

Found 2196016 word vectors


In [20]:
def target_to_one_hot(target, num_classes=3):
    return to_categorical(target, num_classes=num_classes)

In [21]:
def text_to_array(text, article_length=500):
    empty_emb = np.zeros(300)                   # each word is represented by a length 300 vector
    text = text[:-1].split()[:article_length]   # each article is length 500
    
    # look for word embedding, return zero array otherwise. 
    embeds = [embeddings_index.get(x, empty_emb) for x in text]
    embeds += [empty_emb] * (article_length - len(embeds))
    return np.array(embeds)

In [27]:
def batch_gen(train_df, batch_size=64, article_length=500, num_classes=3):
    n = math.ceil(len(train_df) / batch_size)
    while True: 
        train_df = train_df.sample(frac=1.0)
        
        for i in range(n):
            texts = train_df['clean_articles'][i*batch_size: (i+1)*batch_size]
            targets = train_df['targets'][i*batch_size: (i+1)*batch_size]
            
            targets = np.array([target_to_one_hot(t, num_classes) for t in targets])
            text_arr = np.array([text_to_array(text, article_length=article_length) for text in texts])
            yield text_arr, targets

## Setup Model

In [23]:
ARTICLE_LENGTH = 500
BATCH_SIZE = 64
DROPOUT = 0.1
REC_DROPOUT = 0.1

In [24]:
# witheld for validation
 
x_test = np.array([text_to_array(x, article_length=ARTICLE_LENGTH) \
                          for x in tqdm(test_df["clean_articles"])])
y_test = np.array([target_to_one_hot(t) for t in tqdm(test_df["targets"])])

100%|██████████| 522/522 [00:00<00:00, 60797.14it/s]


In [25]:
# note...
#      batch_size         -> words per batch
#      article_length     -> words per article
#      embed_length       -> vector length per word

input_shape = (ARTICLE_LENGTH, 300)
lstm_in = int(BATCH_SIZE/2)

model = Sequential()
model.add(Bidirectional(LSTM(lstm_in, return_sequences=False, \
                        dropout=DROPOUT, recurrent_dropout=REC_DROPOUT), \
                        input_shape=input_shape))
model.add(Activation('relu'))
# model.add(Bidirectional(LSTM(lstm_in)))
model.add(Dense(3, activation='softmax'))
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional_1 (Bidirection (None, 64)                85248     
_________________________________________________________________
activation_1 (Activation)    (None, 64)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 3)                 195       
Total params: 85,443
Trainable params: 85,443
Non-trainable params: 0
_________________________________________________________________


In [28]:
data = batch_gen(train_df, batch_size=BATCH_SIZE, article_length=ARTICLE_LENGTH)
model.fit_generator(data, epochs=2, steps_per_epoch=500, verbose=True)

Epoch 1/2
500/500 [==============================] - 364s 727ms/step - loss: 0.4208 - acc: 0.8393
Epoch 2/2
500/500 [==============================] - 359s 718ms/step - loss: 0.1690 - acc: 0.9415


In [29]:
y_pred = model.predict(x_test, batch_size=BATCH_SIZE)
y_pred_class = np.argmax(y_pred, axis=1)
y_pred_one_hot = to_categorical(y_pred_class, num_classes=3)

In [30]:
y_pred[0:5]

array([[  8.88640702e-01,   1.08859196e-01,   2.50014453e-03],
       [  9.99586936e-03,   2.20355950e-03,   9.87800598e-01],
       [  1.00982422e-02,   9.89862919e-01,   3.88756453e-05],
       [  9.98584032e-01,   7.99316331e-04,   6.16667385e-04],
       [  9.91307318e-01,   8.12769495e-03,   5.65017574e-04]], dtype=float32)

In [31]:
test_df[['source', 'clean_articles', 'targets']].head()

source                                     clean_articles  targets
2990    PBS  The 24-hour news cycle is filled with politics...        0
411     Fox  Pelosi works on Democratic leadership term-lim...        2
1209    Vox  In an interview that aired Sunday on 60 Minute...        1
1925    PBS    White House press secretary Sarah Huckabee S...        0
3166    PBS  President Donald Trump says he has a “great lo...        0

In [33]:
res = metrics.f1_score(y_test, y_pred_one_hot, average='micro')
print("f1 score {res}".format(res=res))

f1 score 0.9425287356321839


# Second dataset
#### Reduce to min (i.e. fox) for balance

In [34]:
df_all.groupby('source').count()

article id  clean_articles  targets
source                                     
Fox            661             661      661
PBS           1739            1739     1739
Vox           1027            1027     1027

## Add more Fox and Vox

In [36]:
# start with the smallest dataset and 
# sample from larger sets
len_fox = len(fox)

df_second = fox.copy()
df_second = df_second.append(pbs.sample(len_fox), ignore_index=True)
df_second = df_second.append(vox.sample(len_fox), ignore_index=True)
df_second.groupby('source').count()

article id  clean_articles  targets
source                                     
Fox            661             661      661
PBS            661             661      661
Vox            661             661      661

## Split into test and training

In [37]:
train_df, test_df = train_test_split(df_second, test_size=0.1)
train_df.head()

article id source  \
1968   vox_politics_398    Vox   
204    fox_politics_551    Fox   
1293  pbs_politics_1182    PBS   
270    fox_politics_235    Fox   
446    fox_politics_325    Fox   

                                         clean_articles  targets  
1968  Senate Republican leaders are postponing a vot...        1  
204   TSA: Airport screener call-outs have risen dur...        2  
1293    Facing a critical Senate vote on his tax ove...        0  
270   Alexandria Ocasio-Cortez thanks supporters in ...        2  
446   Democrat Rep. Ilhan Omar is appointed to the H...        2

In [38]:
test_df.head()

article id source  \
439    fox_politics_804    Fox   
1516  vox_politics_1521    Vox   
1455  vox_politics_1132    Vox   
626    fox_politics_826    Fox   
1333  vox_politics_1172    Vox   

                                         clean_articles  targets  
439   Lanny Davis corrects Cohen-Trump Tower story\n...        2  
1516  A few days before the Wisconsin primary electi...        1  
1455  The Trump administration is finally playing ba...        1  
626   In this Aug. 15, 2018 file photo, Democratic N...        2  
1333  The day before Senate Republicans voted to pas...        1

### Check for similarity between test and training

In [39]:
test_df.groupby('source').count()

article id  clean_articles  targets
source                                     
Fox             62              62       62
PBS             66              66       66
Vox             71              71       71

In [40]:
train_df.groupby('source').count()

article id  clean_articles  targets
source                                     
Fox            599             599      599
PBS            595             595      595
Vox            590             590      590

## Setup Model

In [41]:
from tqdm import tqdm
import numpy as np

In [42]:
ARTICLE_LENGTH = 500
BATCH_SIZE = 64
DROPOUT = 0.1
REC_DROPOUT = 0.1

In [43]:
# witheld for validation
 
x_test = np.array([text_to_array(x, article_length=ARTICLE_LENGTH) \
                          for x in tqdm(test_df["clean_articles"])])
y_test = np.array([target_to_one_hot(t) for t in tqdm(test_df["targets"])])

100%|██████████| 199/199 [00:00<00:00, 48739.65it/s]


In [44]:
# note...
#      batch_size         -> words per batch
#      article_length     -> words per article
#      embed_length       -> vector length per word

input_shape = (ARTICLE_LENGTH, 300)
lstm_in = int(BATCH_SIZE/2)

model = Sequential()
model.add(Bidirectional(LSTM(lstm_in, return_sequences=False, \
                        dropout=DROPOUT, recurrent_dropout=REC_DROPOUT), \
                        input_shape=input_shape))
model.add(Activation('relu'))
# model.add(Bidirectional(LSTM(lstm_in)))
model.add(Dense(3, activation='softmax'))
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional_2 (Bidirection (None, 64)                85248     
_________________________________________________________________
activation_2 (Activation)    (None, 64)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 3)                 195       
Total params: 85,443
Trainable params: 85,443
Non-trainable params: 0
_________________________________________________________________


In [45]:
data = batch_gen(train_df, batch_size=BATCH_SIZE, article_length=ARTICLE_LENGTH)
model.fit_generator(data, epochs=2, steps_per_epoch=500, verbose=True)

Epoch 1/2
500/500 [==============================] - 365s 731ms/step - loss: 0.3831 - acc: 0.8581
Epoch 2/2
500/500 [==============================] - 358s 716ms/step - loss: 0.0752 - acc: 0.9815


In [46]:
y_pred = model.predict(x_test, batch_size=BATCH_SIZE)
y_pred_class = np.argmax(y_pred, axis=1)
y_pred_one_hot = to_categorical(y_pred_class, num_classes=3)

In [47]:
y_pred[0:5]

array([[  1.14628288e-04,   5.67048119e-05,   9.99828696e-01],
       [  5.92374745e-05,   9.99938130e-01,   2.65008612e-06],
       [  6.87164837e-04,   9.99221802e-01,   9.09609080e-05],
       [  7.48341903e-04,   7.18804356e-03,   9.92063582e-01],
       [  1.20473771e-04,   9.99873996e-01,   5.47559466e-06]], dtype=float32)

In [48]:
test_df[['source', 'clean_articles', 'targets']].head()

source                                     clean_articles  targets
439     Fox  Lanny Davis corrects Cohen-Trump Tower story\n...        2
1516    Vox  A few days before the Wisconsin primary electi...        1
1455    Vox  The Trump administration is finally playing ba...        1
626     Fox  In this Aug. 15, 2018 file photo, Democratic N...        2
1333    Vox  The day before Senate Republicans voted to pas...        1

In [49]:
res = metrics.f1_score(y_test, y_pred_one_hot, average='micro')
print("f1 score {res}".format(res=res))

f1 score 0.8341708542713568
